In [428]:
from PIL import Image
import pytesseract as pt
import cv2
import matplotlib.pyplot as plt
import numpy as np
from math import *
from random import *

In [7]:
import argparse
import base64
import getopt
import MultipartPostHandler
import os
import re
import sys
import time
import urllib2
import urllib
import xml.dom.minidom

class ProcessingSettings:
	Language = "English"
	OutputFormat = "txt"


class Task:
	Status = "Unknown"
	Id = None
	DownloadUrl = None
	def IsActive( self ):
		if self.Status == "InProgress" or self.Status == "Queued":
			return True
		else:
			return False

class AbbyyOnlineSdk:
	ServerUrl = "http://cloud.ocrsdk.com/"
	# To create an application and obtain a password,
	# register at http://cloud.ocrsdk.com/Account/Register
	# More info on getting your application id and password at
	# http://ocrsdk.com/documentation/faq/#faq3
	ApplicationId = "Integra+"
	Password = "J0jNchAw53cXMPzhSUg+SkRj "
	Proxy = None
	enableDebugging = 0

	def ProcessImage( self, filePath, settings ):
		urlParams = urllib.urlencode({
			"language" : settings.Language,
			"exportFormat" : settings.OutputFormat
			})
		requestUrl = self.ServerUrl + "processImage?" + urlParams

		bodyParams = { "file" : open( filePath, "rb" )  }
		request = urllib2.Request( requestUrl, None, self.buildAuthInfo() )
		response = self.getOpener().open(request, bodyParams).read()
		if response.find( '<Error>' ) != -1 :
			return None
		# Any response other than HTTP 200 means error - in this case exception will be thrown

		# parse response xml and extract task ID
		task = self.DecodeResponse( response )
		return task

	def GetTaskStatus( self, task ):
		if task.Id.find( '00000000-0' ) != -1 :
			#GUID_NULL is being passed. This may be caused by a logical error in the calling code
			print "Null task id passed"
			return None

		urlParams = urllib.urlencode( { "taskId" : task.Id } )
		statusUrl = self.ServerUrl + "getTaskStatus?" + urlParams
		request = urllib2.Request( statusUrl, None, self.buildAuthInfo() )
		response = self.getOpener().open( request ).read()
		task = self.DecodeResponse( response )
		return task

	def DownloadResult( self, task, outputPath ):
		getResultUrl = task.DownloadUrl
		if getResultUrl == None :
			print "No download URL found"
			return
		request = urllib2.Request( getResultUrl )
		fileResponse = self.getOpener().open( request ).read()
		resultFile = open( outputPath, "wb" )
		resultFile.write( fileResponse )


	def DecodeResponse( self, xmlResponse ):
		""" Decode xml response of the server. Return Task object """
		dom = xml.dom.minidom.parseString( xmlResponse )
		taskNode = dom.getElementsByTagName( "task" )[0]
		task = Task()
		task.Id = taskNode.getAttribute( "id" )
		task.Status = taskNode.getAttribute( "status" )
		if task.Status == "Completed":
			task.DownloadUrl = taskNode.getAttribute( "resultUrl" )
		return task


	def buildAuthInfo( self ):
		return { "Authorization" : "Basic %s" % base64.b64encode( "%s:%s" % (self.ApplicationId, self.Password) ) }

	def getOpener( self ):
		if self.Proxy == None:
			self.opener = urllib2.build_opener( MultipartPostHandler.MultipartPostHandler,
			urllib2.HTTPHandler(debuglevel=self.enableDebugging))
		else:
			self.opener = urllib2.build_opener( 
				self.Proxy, 
				MultipartPostHandler.MultipartPostHandler,
				urllib2.HTTPHandler(debuglevel=self.enableDebugging))
		return self.opener

In [48]:
first_task= Task()
t= ProcessingSettings()
ab_1=AbbyyOnlineSdk()
filePath='/Users/assansanogo/Downloads/these_pictures/clonotril_clonazepam_2.jpg'


first_task=ab_1.ProcessImage(filePath, t)

In [49]:
stat_1=ab_1.GetTaskStatus(first_task)

In [50]:
stat_1

<__main__.Task instance at 0x10b527dd0>

In [51]:
outputPath='/Users/assansanogo/Downloads/these_pictures/Q_ocr.txt'

In [52]:
ab_1.DownloadResult( stat_1, outputPath )

In [53]:
l=[]
with open(outputPath,'r') as f:

    l=f.read().split("\r")
l2=list(map(str.strip,l))


In [54]:
l2

['\xef\xbb\xbfCLONOTRIL-2',
 '',
 '',
 '',
 '10 STRIPS OF 10 TABLETS EACH',
 '',
 'Manutaci by',
 'TORRENT PHARMACEUTICALS LTD',
 '3? No M oale Cam\xc2\xa9 NM-31A.',
 "East DtV'iCt. Ganqto. Sik\xc2\xabim-737 135.",
 '']

In [55]:
def remove_unicode(x):
    r=re.sub(r'[^\x00-\x7F]+','', x)
    return r

In [56]:
l2=list(map(remove_unicode,l2))
l2=[el for el in l2 if len(el)!=0]

In [57]:
l2

['CLONOTRIL-2',
 '10 STRIPS OF 10 TABLETS EACH',
 'Manutaci by',
 'TORRENT PHARMACEUTICALS LTD',
 '3? No M oale Cam NM-31A.',
 "East DtV'iCt. Ganqto. Sikim-737 135."]